## take a list of cities

In [9]:
import math
import folium
from dotenv import load_dotenv
from os import getenv

import requests
import json

import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.distance import ELLIPSOIDS, distance

from datetime import datetime
import sqlite3
import random

In [10]:
load_dotenv()
API_KEY = getenv("MAPILLARY_CLIENT_TOKEN")

test_dir = "d://projects_working_directories//202408_pano_images"
target_cities = "d://projects_working_directories//202408_pano_images//target_cities.xlsx"

df_target_cities = pd.read_excel(target_cities)
df_target_cities['city_key'] = df_target_cities.country + " | " + df_target_cities.city
df_target_cities

,country,city,lat,lon,bbox,city_key
0,Indonesia,Jakarta,-6.222932,106.833616,50,Indonesia | Jakarta
1,Indonesia,Surabaya,-7.277315,112.729501,50,Indonesia | Surabaya


In [11]:
def get_bounding_box(lat, lon, extend_lat_meters, extend_lon_meters):
 # Earth radius in meters
    R = 6378137
    
    # Coordinate offsets in radians
    dLat = extend_lat_meters / R
    dLon = extend_lon_meters / (R * math.cos(math.pi * lat / 180))
    
    # Offset in degrees
    lat_min = lat - dLat * 180 / math.pi
    lat_max = lat + dLat * 180 / math.pi
    lon_min = lon - dLon * 180 / math.pi
    lon_max = lon + dLon * 180 / math.pi


    
    return [lat_min, lat_max, lon_min, lon_max]







def generate_random_coordinates(bbox, num_samples):

    lat_min = bbox[0]
    lat_max = bbox[1]
    lon_min = bbox[2]
    lon_max = bbox[3]
    
    random_coords = []
    for _ in range(num_samples):
        lat = random.uniform(lat_min, lat_max)
        lon = random.uniform(lon_min, lon_max)
        random_coords.append((lat, lon))
    return random_coords







def display_map(source_lat_lon, bbox, points=None):

    lat_min = bbox[0]
    lat_max = bbox[1]
    lon_min = bbox[2]
    lon_max = bbox[3]

    
    # Create a map centered around the given coordinates
    m = folium.Map(location=source_lat_lon, zoom_start=11)
    
    # Define the bounding box coordinates
    bounds = [
        [lat_min, lon_min],
        [lat_max, lon_min],
        [lat_max, lon_max],
        [lat_min, lon_max],
        [lat_min, lon_min]  # Close the polygon
    ]
    
    # Add the bounding box to the map
    folium.PolyLine(bounds, color="blue", weight=2.5, opacity=1).add_to(m)

    # Add the center point to the map
    folium.Marker(location=source_lat_lon, popup="Center", icon=folium.Icon(color="red")).add_to(m)
    
    # Add the points to the map
    if points:
        for point in points:
            folium.CircleMarker(location=point, radius=3, color='darkred', fill=True, fill_color='darkred').add_to(m)
        
    return m

#map_with_bounding_box = display_map(source_lat_lon, bbox, points=sample_coordinates)
#map_with_bounding_box.save("map_with_bounding_box.html")

# Display the map
#map_with_bounding_box

In [12]:
def get_closest_image_id_from_coordinates(access_token, latitude: int, longitude: int) -> dict:
    #log.debug("Get Image From Coordinates: %s, %s", latitude, longitude, extend_lat_meters=10, extend_lon_meters=10 )
    results = {
        "image_lat": None,
        "image_lon": None,
        "residual": None,
        "image_id": None,
        "camera_type": None,
        "is_pano": None,
        "camera_focal_len": None,
        "camera_k1": None,
        "camera_k2": None,
        "image_path": None,
        "error": None,
    }


    bbox = get_bounding_box(latitude, longitude, 200, 200)
    lat_min = bbox[0]
    lat_max = bbox[1]
    lon_min = bbox[2]
    lon_max = bbox[3]

    mapillary_bbox = f"{lon_min},{lat_min},{lon_max},{lat_max}"



    url = "https://graph.mapillary.com/images"
    try:
        response = requests.get(
            url,
            params={
                "access_token": access_token,
                "fields": "id,thumb_original_url,geometry,camera_type,camera_parameters,is_pano",
                #"fields": "id,thumb_2048_url,geometry",
                #"is_pano": "true",
                "bbox": mapillary_bbox
            },
        )
        response.raise_for_status()
    except:
        response.raise_for_status()
        print("error")
        return results

    images = response.json()["data"]
    #log.debug("Successfully Retrieved Image Data: %s", images)
    if len(images) == 0:
        #log.debug(
        #    "No Images in Bounding Box: %s", self._bounds(latitude, longitude)
        #)
        return results

    closest = 0
    closest_distance = np.inf

    for i, image in enumerate(images):
    #    filter(lambda img: img["id"] not in self.downloaded_images, images)
    #):
        #print(image)
        image_coordinates = (
            image["geometry"]["coordinates"][1],
            image["geometry"]["coordinates"][0],
        )
        residual = distance(
            (latitude, longitude), image_coordinates, ellipsoid=ELLIPSOIDS["WGS-84"]
        )
        if residual < closest_distance:
            closest = i
            closest_distance = residual

    image = images[closest]
    #log.debug("Closest Image: %s", image["id"])
    results["image_id"] = image["id"]
    results["image_lat"] = image["geometry"]["coordinates"][1]
    results["image_lon"] = image["geometry"]["coordinates"][0]
    results["residual"] = closest_distance.m
    results["image_url"] = image["thumb_original_url"]
    results["camera_type"] = image["camera_type"]
    results["is_pano"] = image["is_pano"]

    camera_params = image.get("camera_parameters")
    if camera_params is not None:
    
        results["camera_focal_len"] = camera_params[0]
        results["camera_k1"] = camera_params[1]
        results["camera_k2"] = camera_params[2]



    return results

def get_image_ids_from_coordinates(access_token, latitude: int, longitude: int):

    df = pd.DataFrame(columns = ['image_lat','image_lon','residual','image_id','camera_type','is_pano','camera_focal_len','camera_k1','camera_k2','image_path','error','image_url','altitude','compass_angle','exif_orientation','make','model'])
    
    #log.debug("Get Image From Coordinates: %s, %s", latitude, longitude, extend_lat_meters=10, extend_lon_meters=10 )
    results = {
        "image_lat": None,
        "image_lon": None,
        "residual": None,
        "image_id": None,
        "camera_type": None,
        "is_pano": None,
        "camera_focal_len": None,
        "camera_k1": None,
        "camera_k2": None,
        "image_path": None,
        "error": None,
        "altitude": None,
        "compass_angle": None,
        "exif_orientation": None,
        "make": None,
        "model": None
    }


    bbox = get_bounding_box(latitude, longitude, 200, 200)
    lat_min = bbox[0]
    lat_max = bbox[1]
    lon_min = bbox[2]
    lon_max = bbox[3]

    mapillary_bbox = f"{lon_min},{lat_min},{lon_max},{lat_max}"



    url = "https://graph.mapillary.com/images"
    try:
        response = requests.get(
            url,
            params={
                "access_token": access_token,
                "fields": "id,thumb_original_url,geometry,camera_type,camera_parameters,is_pano,altitude,compass_angle,exif_orientation,make,model",
                #"fields": "id,thumb_2048_url,geometry",
                "is_pano": "true",
                "limit": 1,
                "bbox": mapillary_bbox
            },
        )
        response.raise_for_status()
    except:
        response.raise_for_status()
        print("error")
        return df

    images = response.json()["data"]
    #log.debug("Successfully Retrieved Image Data: %s", images)
    if len(images) == 0:
        #log.debug(
        #    "No Images in Bounding Box: %s", self._bounds(latitude, longitude)
        #)
        return df

    closest = 0
    closest_distance = np.inf

    for i, image in enumerate(images):
    #    filter(lambda img: img["id"] not in self.downloaded_images, images)
    #):
        print(image)
        image_coordinates = (
            image["geometry"]["coordinates"][1],
            image["geometry"]["coordinates"][0],
        )
        residual = distance(
            (latitude, longitude), image_coordinates, ellipsoid=ELLIPSOIDS["WGS-84"]
        )
        #if residual < closest_distance:
          #  closest = i
         #   closest_distance = residual

        #image = images[closest]
        #log.debug("Closest Image: %s", image["id"])
        results["image_id"] = image["id"]
        results["image_lat"] = image["geometry"]["coordinates"][1]
        results["image_lon"] = image["geometry"]["coordinates"][0]
        results["residual"] = residual.m
        results["image_url"] = image.get("thumb_original_url")
        results["camera_type"] = image.get("camera_type")
        results["is_pano"] = image.get("is_pano")


        results["altitude"] = image.get("altitude")
        results["compass_angle"] = image.get("compass_angle")
        results["exif_orientation"] = image.get("exif_orientation")
        results["make"] = image.get("make")
        results["model"] = image.get("model")
       
    
        camera_params = image.get("camera_parameters")
        if camera_params is not None:
        
            results["camera_focal_len"] = camera_params[0]
            results["camera_k1"] = camera_params[1]
            results["camera_k2"] = camera_params[2]
        #else:
        #    results["camera_focal_len"] = 'unavailable'
        #    results["camera_k1"] = 'unavailable'
        #    results["camera_k2"] = 'unavailable'
  
        df.loc[len(df)] = pd.Series(results)
        #tmp_df = pd.DataFrame([results])
        
        #df = pd.concat([df,tmp_df], ignore_index=True)

    if len(df) > 0:
        return df
    return None

In [13]:

df_all = pd.DataFrame()


for i, row in df_target_cities.iterrows():
    source_latitude = row.lat
    source_longitude = row.lon
    # this will create a square where each side is 2x the meters
    extend_lat_meters = 20_000 
    extend_lon_meters = 20_000 
    
    num_samples = 10 # specify the number of random samples you want

    bbox = get_bounding_box(source_latitude, source_longitude, extend_lat_meters, extend_lon_meters)
    sample_coordinates = generate_random_coordinates(bbox, num_samples)

    
    df = pd.DataFrame(columns = ['image_lat','image_lon','residual','image_id','camera_type',
    'is_pano','camera_focal_len','camera_k1','camera_k2','image_path','error','image_url',
    'altitude','compass_angle','exif_orientation','make','model'])
    
    for coords in sample_coordinates:
     
        
        tmp_df = get_image_ids_from_coordinates(API_KEY, coords[0], coords[1])
        if isinstance(tmp_df, pd.DataFrame):
            df = pd.concat([df,tmp_df], ignore_index=True)

    df['country'] = row.country
    df['city'] = row.city

    df_all = pd.concat([df_all,df])
    

{'id': '1505018570392136', 'thumb_original_url': 'https://scontent.ffsd3-1.fna.fbcdn.net/m1/v/t6/An_BsALpAV6f4DjjTTJtnYEmWGn0qyiZBfqTLBnouxJf8qVF3EXPd4t_dkoFEDG2Yh70iVl5ffChtFrDOEdpPfQK0HnHdO_7pcciHxWJF1NhFRk2YxMIsvOt_8xq2WFpYOar0McXKsL_A4gYBiG-ag?ccb=10-5&oh=00_AYBN0sl2UXdDmOtKX2Vqnq3hIo5ASmzhY3ZOXsv9xyhUsQ&oe=66FDBD62&_nc_sid=201bca', 'geometry': {'type': 'Point', 'coordinates': [106.7843904, -6.2633539]}, 'camera_type': 'spherical', 'is_pano': True, 'altitude': 48.755, 'compass_angle': 262.83362725926, 'exif_orientation': 1, 'make': 'GoPro', 'model': 'GoPro Max'}
{'id': '1095545924696245', 'thumb_original_url': 'https://scontent.ffsd3-1.fna.fbcdn.net/m1/v/t6/An9BXK5mtDvCgjbmUTy1wTsPT-jZ52uqemltQp7fSb1M1ShfnsyEuNrmec4hqZrK6J4q4e7u8zH-tpDIMGYQJxyIjVlZIn90bjRyYF1Tkiy9-_cvGzZWewrWw4UN1yxZw86z4_qAXTQKJkVVthCURWc?ccb=10-5&oh=00_AYDCJxd1xmk-ndCM_iZELDbkYcJ0s0bMzge9ksxHDa8eeA&oe=66FDA6B9&_nc_sid=201bca', 'geometry': {'type': 'Point', 'coordinates': [112.75010779997, -7.4266151]}, 'camera_ty

In [14]:
df_all

,image_lat,image_lon,residual,image_id,camera_type,is_pano,camera_focal_len,camera_k1,camera_k2,image_path,error,image_url,altitude,compass_angle,exif_orientation,make,model,country,city
0,-6.263354,106.78439,177.236655,1505018570392136,spherical,True,None,None,None,None,None,https://scontent.ffsd3-1.fna.fbcdn.net/m1/v/t6...,48.755,262.833627,1,GoPro,GoPro Max,Indonesia,Jakarta
0,-7.426615,112.750108,140.153,1095545924696245,spherical,True,None,None,None,None,None,https://scontent.ffsd3-1.fna.fbcdn.net/m1/v/t6...,32.769,0,1,GoPro,GoPro Max,Indonesia,Surabaya
1,-7.273224,112.638557,217.079601,1564726257278323,spherical,True,None,None,None,None,None,https://scontent.ffsd3-1.fna.fbcdn.net/m1/v/t6...,58.031,0,1,GoPro,GoPro Max,Indonesia,Surabaya


In [15]:
print(f"before dedupe {df_all.shape}")
df_all = df_all.drop_duplicates().reset_index(drop=True)
print(f"after dedupe {df_all.shape}")


before dedupe (3, 19)
after dedupe (3, 19)


In [16]:
timestamp = datetime.now().strftime('%Y-%m-%d')

conn = sqlite3.connect(f'{test_dir}//360_images_{timestamp}.db')


df_all.to_sql('sampled_images', con=conn, if_exists='replace', index=False)



# Close the connection
conn.close()